### Imports

In [1]:
import torch
import torch.nn as nn 
import torch.optim as optim
import torch.nn.functional as F 
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

### Create simple CNN Network

In [2]:
class CNN(nn.Module):
    
    def __init__(self, in_channels=1, num_classes=10):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.fc1 = nn.Linear(16*7*7, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)

        return x

### Testing Code

- 64 random images with size (28x28) = 784

In [3]:
img = torch.randn(64, 1, 28, 28)

- check image size

In [4]:
img.shape

torch.Size([64, 1, 28, 28])

In [5]:
img[0].shape

torch.Size([1, 28, 28])

- initalize model, which takes input `784` and outputs `10` tensors

In [6]:
model = CNN()

In [7]:
print(model(img).shape)

torch.Size([64, 10])


### Set Device

In [8]:
device = "cpu"

if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"

In [9]:
device = torch.device(device)
device

device(type='mps')

### Hyperparameters

In [10]:
in_channes = 1
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 5

### Load Data

In [11]:
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=False)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

### Initalize Networks

In [12]:
model = CNN().to(device)

### Loss and Optimizers

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

### Train Network

In [14]:
for epoch in range(num_epochs):

    total_loss = 0

    for batch_idx, (data, target) in enumerate(train_loader):

        data = data.to(device)
        target = target.to(device)

        # data = data.reshape(data.shape[0], -1) # (64, 1, 28, 28) --> (64, 1x28x28) = (64, 784)

        # forward pass
        scores = model(data) # Forward pass
        
        loss = criterion(scores, target) # Compute loss

        # backward
        optimizer.zero_grad() # Clear previous gradients

        loss.backward() # Backpropagation

        optimizer.step() # Update model weights

        total_loss = loss.item() # Accumulate loss

    average_loss = total_loss / len(train_loader) # Calculating average loss

    print(f"Epoch {epoch}: Average Loss: {average_loss}")

Epoch 0: Average Loss: 0.00011006933349027816
Epoch 1: Average Loss: 8.116461543131993e-05
Epoch 2: Average Loss: 2.1233155862736042e-05
Epoch 3: Average Loss: 7.043303901961109e-05
Epoch 4: Average Loss: 0.00013836211105907903


### Check accuracy on train & test see how good our model

In [15]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("CHecking accuarcy on testing data")

    num_correct = 0
    num_samples = 0
    model.eval() # put model into evaluation mode

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            scores = model(x)

            _, predictions = scores.max(1)

            num_correct += (predictions == y).sum()

            num_samples += predictions.size(0)
        acc = float(num_correct) / float(num_samples) * 100

        print(f"Got {num_correct} / {num_samples} with accuracy {acc:.2f}")
    
    # model.train()

In [16]:
check_accuracy(train_loader, model)

Checking accuracy on training data
Got 59203 / 60000 with accuracy 98.67


In [17]:
check_accuracy(test_loader, model)

CHecking accuarcy on testing data
Got 9839 / 10000 with accuracy 98.39
